# US Airport Delays

This notebook will analyze the reasons for flight cancellations and delays in 2008 

[Airline Codes](http://stat-computing.org/dataexpo/2009/carriers.csv), [Airport Codes](http://stat-computing.org/dataexpo/2009/airports.csv), [Data Dictionary](http://stat-computing.org/dataexpo/2009/the-data.html)

In [109]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
import statsmodels.formula.api as smf
import math
import sys
print sys.version
from matplotlib import style
import urllib
#import bz2file # so we can dowload directly from the website instead of saving the large csv files locally
import bz2
style.use('fivethirtyeight')
%pylab inline

2.7.11 (default, Dec  5 2015, 14:44:47) 
[GCC 4.2.1 Compatible Apple LLVM 7.0.0 (clang-700.1.76)]
Populating the interactive namespace from numpy and matplotlib


In [110]:
# # opens from link, but does not currently work
# filename = 'datasets/temp.csv'  
# req = urllib.urlopen('http://stat-computing.org/dataexpo/2009/2006.csv.bz2') # change year for each file
# CHUNK = 16 * 1024

# decompressor = bz2.BZ2Decompressor()
# with open(filename, 'wb') as fp:
#     while True:
#         chunk = req.read(CHUNK)
#         if not chunk:
#             break
#         fp.write(decompressor.decompress(chunk))
# req.close()
# # open the 'temp' file
# flights = pd.read_csv('temp.csv')

### Set year variable below:

In [111]:
year = 1996

In [112]:
# open downloaded csv file with pandas
flights = pd.read_csv('datasets/flight_hist_raw/%d.csv.bz2' % year)

In [113]:
# open the 'temp' file
#flights = pd.read_csv('temp.csv')

In [114]:
flights.head()

,Year,Month,DayofMonth,DayOfWeek,DepTime,CRSDepTime,ArrTime,CRSArrTime,UniqueCarrier,FlightNum,...,TaxiIn,TaxiOut,Cancelled,CancellationCode,Diverted,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay
0,1996,1,29,1,2039,1930,2245,2139,DL,345,...,6,10,0,NaN,0,NaN,NaN,NaN,NaN,NaN
1,1996,1,30,2,1931,1930,2142,2139,DL,345,...,5,22,0,NaN,0,NaN,NaN,NaN,NaN,NaN
2,1996,1,31,3,1956,1930,2231,2139,DL,345,...,7,27,0,NaN,0,NaN,NaN,NaN,NaN,NaN
3,1996,1,1,1,1730,1550,1909,1745,DL,411,...,4,14,0,NaN,0,NaN,NaN,NaN,NaN,NaN
4,1996,1,2,2,1714,1550,1841,1745,DL,411,...,4,8,0,NaN,0,NaN,NaN,NaN,NaN,NaN


In [115]:
# create lane pairs to determine highest traffic routes
flights['lanes'] = flights['Origin'] + ' - ' + flights['Dest']

# # create two empty lists to store origin airports and destination airports
# origin = []
# dest = []

# def split_col(series):
#     for i in series:
#         s = i.split('-') # split each pair on the dash
#         origin.append(s[0]) # append the first value to origin
#         dest.append(s[1]) # append the second value to destination
        
# # run split function on lane pair column in the merged dataframe
# split_col(flights['lanes'])

# # create new columns in merged df
# merged['Origin'] = np.asarray(origin)
# merged['Dest'] = np.asarray(dest)
# merged.head()

# next, let's read in some data on the airports and air carriers 
carriers = pd.read_csv('http://stat-computing.org/dataexpo/2009/carriers.csv')
airports = pd.read_csv('http://stat-computing.org/dataexpo/2009/airports.csv')

airports_orig = pd.DataFrame(airports[['iata','airport']])
airports_orig.rename(columns = {'iata':'Origin'}, inplace = True)

flights2 = pd.merge(flights,airports_orig, on ='Origin')
flights2.rename(columns = {'airport':'origin_airport'}, inplace = True)
flights2.columns

airports_dest = pd.DataFrame(airports[['iata','airport']])
airports_dest.rename(columns = {'iata':'Dest'}, inplace = True)

flights3 = pd.merge(flights2,airports_dest, on ='Dest')
flights3.rename(columns = {'airport':'dest_airport'}, inplace = True)

# then create origin-destination pairs
flights3['lane_pairs'] = flights3['origin_airport'] + '-' + flights3['dest_airport']

In [116]:
flights3.columns

Index([u'Year', u'Month', u'DayofMonth', u'DayOfWeek', u'DepTime',
       u'CRSDepTime', u'ArrTime', u'CRSArrTime', u'UniqueCarrier',
       u'FlightNum', u'TailNum', u'ActualElapsedTime', u'CRSElapsedTime',
       u'AirTime', u'ArrDelay', u'DepDelay', u'Origin', u'Dest', u'Distance',
       u'TaxiIn', u'TaxiOut', u'Cancelled', u'CancellationCode', u'Diverted',
       u'CarrierDelay', u'WeatherDelay', u'NASDelay', u'SecurityDelay',
       u'LateAircraftDelay', u'lanes', u'origin_airport', u'dest_airport',
       u'lane_pairs'],
      dtype='object')

In [117]:
# adding origin and destination state, airport codes
airports.head()

,iata,airport,city,state,country,lat,long
0,00M,Thigpen,Bay Springs,MS,USA,31.953765,-89.234505
1,00R,Livingston Municipal,Livingston,TX,USA,30.685861,-95.017928
2,00V,Meadow Lake,Colorado Springs,CO,USA,38.945749,-104.569893
3,01G,Perry-Warsaw,Perry,NY,USA,42.741347,-78.052081
4,01J,Hilliard Airpark,Hilliard,FL,USA,30.688012,-81.905944


In [118]:
# add origin state
orig_state = airports.rename(columns = {'iata':'Origin'})
orig_state2 = pd.merge(orig_state, flights3, on = 'Origin')
orig_state2.rename(columns = {'state':'orig_state'}, inplace = True)
#orig_state2.columns


# add destination state
dest_state = airports.rename(columns = {'iata':'Dest'})
dest_state2 = pd.merge(orig_state2,dest_state, on = 'Dest')
dest_state2.rename(columns = {'state':'dest_state'}, inplace = True)

dest_state2.columns

Index([u'Origin', u'airport_x', u'city_x', u'orig_state', u'country_x',
       u'lat_x', u'long_x', u'Year', u'Month', u'DayofMonth', u'DayOfWeek',
       u'DepTime', u'CRSDepTime', u'ArrTime', u'CRSArrTime', u'UniqueCarrier',
       u'FlightNum', u'TailNum', u'ActualElapsedTime', u'CRSElapsedTime',
       u'AirTime', u'ArrDelay', u'DepDelay', u'Dest', u'Distance', u'TaxiIn',
       u'TaxiOut', u'Cancelled', u'CancellationCode', u'Diverted',
       u'CarrierDelay', u'WeatherDelay', u'NASDelay', u'SecurityDelay',
       u'LateAircraftDelay', u'lanes', u'origin_airport', u'dest_airport',
       u'lane_pairs', u'airport_y', u'city_y', u'dest_state', u'country_y',
       u'lat_y', u'long_y'],
      dtype='object')

In [119]:
# rename fully merged dataframe for cleaner replication
flights5 = dest_state2
flights5.columns

Index([u'Origin', u'airport_x', u'city_x', u'orig_state', u'country_x',
       u'lat_x', u'long_x', u'Year', u'Month', u'DayofMonth', u'DayOfWeek',
       u'DepTime', u'CRSDepTime', u'ArrTime', u'CRSArrTime', u'UniqueCarrier',
       u'FlightNum', u'TailNum', u'ActualElapsedTime', u'CRSElapsedTime',
       u'AirTime', u'ArrDelay', u'DepDelay', u'Dest', u'Distance', u'TaxiIn',
       u'TaxiOut', u'Cancelled', u'CancellationCode', u'Diverted',
       u'CarrierDelay', u'WeatherDelay', u'NASDelay', u'SecurityDelay',
       u'LateAircraftDelay', u'lanes', u'origin_airport', u'dest_airport',
       u'lane_pairs', u'airport_y', u'city_y', u'dest_state', u'country_y',
       u'lat_y', u'long_y'],
      dtype='object')

In [120]:
# # first create a bar graph of the types of cancellations by type
# # the four categories are broken down as follows: A = carrier, B = weather, C = NAS, D = security
# c_code = {'CancellationCode':['A','B','C','D']}

# cancellations = flights['FlightNum'].groupby(flights['CancellationCode']).count()
# cancellations = pd.DataFrame(cancellations)
# cancellations.reset_index('CancellationCode', inplace = True)
# cancellations.rename(columns = {0:'cancellation_count'}, inplace = True)
# cancellations


In [121]:
# # calculate the average duration of each delay by type
# # first, carrier delays
# a_delay = flights['CarrierDelay'].mean()
# b_delay = flights['WeatherDelay'].mean()
# c_delay = flights['NASDelay'].mean()
# d_delay = flights['SecurityDelay'].mean()
# l_delay = flights['LateAircraftDelay'].mean()

# delay_dict = {'Carrier' : [a_delay], 
#               'Weather' : [b_delay], 
#               'NAS' : [c_delay], 
#               'Security' : [d_delay], 
#               'Late Plane' : [l_delay]
#              }

# delays = [a_delay, b_delay, c_delay, d_delay]
# delays = pd.DataFrame(delays)
# delays.rename(columns = {0:'avg_delay'}, inplace = True)
# delays['CancellationCode'] = ['A','B','C','D']
# # delays.set_index([['A','B','C','D']], inplace = True)
# # delays.reset_index()
# delays

In [122]:
# # now merge delay counts and average delay dataframes
# cancel_info = pd.merge(delays,cancellations, on = 'CancellationCode')
# cancel_info['category'] = ['Carrier','Weather','NAS','Security']
# cancel_info.set_index('CancellationCode', inplace = True)
# cancel_info.reset_index('CancellationCode', inplace = True)
# cancel_info.rename(columns = {'avg_delay':'avg_delay_min'}, inplace = True) # rename avg_delay to denote minutes
# cancel_info.head()

In [123]:
# # before graphing, set the category to the index for better xlabels
# cancel_info.set_index('category', inplace = True)
# cancel_info.cancellation_count.plot(kind = 'bar')
# plt.xlabel('Cancellation Code')
# plt.ylabel('Number of Flights')
# plt.title('Causes for Flight Cancellations in 2008')

In [124]:
# # since there is no record for cancellations due to late airplaces, we will have to measure the delays separately
# delays = pd.DataFrame.from_dict(delay_dict)
# delays = delays.T
# delays.rename(columns = {0:'avg_delay_min'}, inplace = True)
# delays.head()

In [125]:
# delays['avg_delay_min'].plot(kind = 'bar')
# plt.title('Causes for Flight Delays in 2008')
# plt.xlabel('Causes')
# plt.ylabel('Average Delay in Minutes')

In [126]:
# create a unique value (index-like) for the dataframe by resetting the index, then using that value as a key
# flights.reset_index(inplace = True)

In [127]:
# since counting the number of occurrences of each delay and keeping them in dataframes of the same length is a pain
# i'm going to do that another way: creating a column of 1s whenever there's an occurrence for each delay type

my_list = []

def delay_flag(series):
    for i in series:
        if math.isnan(i) == True or i == 0.0: 
        # use math module to determine if value is nan -> numpy doesn't necessarily work
        # use floats because the dtype is float64 (seen at the beginning)
            my_list.append(0) # return 0 if it's zero or nan
        else:
            my_list.append(1) # otherwise return 1 so we can count them later

In [128]:
# create flag variables for each delay type, then append them to the flights5 dataframe

my_list = []
delay_flag(flights5['CarrierDelay'])
flights5['f_carrier_delay'] = np.asarray(my_list)

my_list = []
delay_flag(flights5['WeatherDelay'])
flights5['f_weather_delay'] = np.asarray(my_list)

my_list = []
delay_flag(flights5['NASDelay'])
flights5['f_nas_delay'] = np.asarray(my_list)

my_list = []
delay_flag(flights5['SecurityDelay'])
flights5['f_sec_delay'] = np.asarray(my_list)

my_list = []
delay_flag(flights5['LateAircraftDelay'])
flights5['f_late_aircraft_delay'] = np.asarray(my_list)

In [129]:
#flights5.columns

In [130]:
# # change my_list to numpy array - TEST

# df = pd.DataFrame(np.random.random_integers(0,25,100))
# df.rename(columns = {0:'test'}, inplace = True)
# df['my_list'] = np.asarray(my_list)
# df.head()

In [131]:
# create lane pairs to determine highest traffic routes
# flights['lanes'] = flights['Origin'] + ' - ' + flights['Dest']

In [132]:
# group number of trips (using flight number) by lane pairs
lane_trips = flights5['FlightNum'].groupby(flights5['lane_pairs']).agg('count')
lane_trips = pd.DataFrame(lane_trips)
lane_trips.rename(columns = {0:'lane_trip_count'}, inplace = True)
lane_trips.reset_index(inplace = True)

print lane_trips.describe()
print ''
print lane_trips.head()

       lane_trip_count
count      3189.000000
mean       1678.263719
std        1849.340103
min           1.000000
25%         541.000000
50%        1095.000000
75%        2181.000000
max       17772.000000

                                 lane_pairs  lane_trip_count
0                     Adams -Chicago Midway              364
1  Adams -Cincinnati Northern Kentucky Intl              708
2                       Adams -Dallas Love              2605
3    Adams -Dallas-Fort Worth International             2191
4                        Adams -Denver Intl               97


In [133]:
# create a stripped down of flights5 with desired columns to merge into lane_df_trips
flights6 = pd.DataFrame(flights5[['lane_pairs',
                     'origin_airport',
                     'orig_state',
                     'dest_airport',
                     'dest_state',]])

In [134]:
#flights6.head()

In [135]:
trips = flights5

In [136]:
# then group lane pairs by each type of delay
# i'm filtering the columns to exclude NaNs instead of values greater than 0 
# because 0 returns each series with a different length, making them difficult to merge

# first we'll do carrier delays
lane_df_carrier_delay = trips['f_carrier_delay'].groupby(trips['lane_pairs']).agg('sum')
lane_df_carrier_delay = pd.DataFrame(lane_df_carrier_delay)
lane_df_carrier_delay.rename(columns = {0:'carrier_delays'}, inplace = True)
lane_df_carrier_delay.reset_index(inplace = True)
#print lane_df_carrier_delay.head()

# then we will do weather delays
lane_df_weather_delay = trips['f_weather_delay'].groupby(trips['lane_pairs']).agg('sum')
lane_df_weather_delay = pd.DataFrame(lane_df_weather_delay)
lane_df_weather_delay.rename(columns = {0:'weather_delays'}, inplace = True)
lane_df_weather_delay.reset_index(inplace = True)
#lane_df_weather_delay.head()

# NAS delays
lane_df_nas_delay = trips['f_nas_delay'].groupby(trips['lane_pairs']).agg('sum')
lane_df_nas_delay = pd.DataFrame(lane_df_nas_delay)
lane_df_nas_delay.rename(columns = {0:'nas_delays'}, inplace = True)
lane_df_nas_delay.reset_index(inplace = True)

# security delays
lane_df_sec_delay = trips['f_sec_delay'].groupby(trips['lane_pairs']).agg('sum')
lane_df_sec_delay = pd.DataFrame(lane_df_sec_delay)
lane_df_sec_delay.rename(columns = {0:'security_delays'}, inplace = True)
lane_df_sec_delay.reset_index(inplace = True)

# late aircraft delays
lane_df_late_delay = trips['f_late_aircraft_delay'].groupby(trips['lane_pairs']).agg('sum')
lane_df_late_delay = pd.DataFrame(lane_df_late_delay)
lane_df_late_delay.rename(columns = {0:'late_aircraft_delays'}, inplace = True)
lane_df_late_delay.reset_index(inplace = True)

print 'length of lane pairs:', len(lane_trips)
print 'length of carrier delays:', len(lane_df_carrier_delay)
print 'length of weather delays:', len(lane_df_weather_delay)
print 'length of NAS delays:', len(lane_df_nas_delay)
print 'length of late plane delays:', len(lane_df_late_delay)


length of lane pairs: 3189
length of carrier delays: 3189
length of weather delays: 3189
length of NAS delays: 3189
length of late plane delays: 3189


## Now that they're all the same length, we can combine them all with the lane pairs

In [137]:
# waterfall merge since pd.merge doesn't like more than merging two dataframes at once
df1 = pd.merge(lane_trips, lane_df_carrier_delay, on = 'lane_pairs')
df2 = pd.merge(df1, lane_df_weather_delay, on = 'lane_pairs')
df3 = pd.merge(df2, lane_df_nas_delay, on = 'lane_pairs')

# change names so they're easier to read
all_delays = pd.merge(df3, lane_df_late_delay, on = 'lane_pairs')
all_delays.rename(columns = {'lane_trip_count': 'trip_count',
                         'f_carrier_delay':'carrier',
                         'f_weather_delay':'weather',
                         'f_nas_delay':'NAS', 
                         'f_late_aircraft_delay':'late_aircraft'
                        }, inplace = True)
#all_delays.head()


In [138]:
merged = pd.DataFrame(pd.merge(flights6, all_delays, on = 'lane_pairs', how = 'right'))

In [139]:
merged = merged.pivot_table(index=['lane_pairs','origin_airport','orig_state','dest_airport','dest_state'])
merged = merged.reset_index()
# add column of current year
merged['year'] = year
merged.ix[0:4]

,lane_pairs,origin_airport,orig_state,dest_airport,dest_state,NAS,carrier,late_aircraft,trip_count,weather,year
0,Adams -Chicago Midway,Adams,AR,Chicago Midway,IL,0,0,0,364,0,1996
1,Adams -Cincinnati Northern Kentucky Intl,Adams,AR,Cincinnati Northern Kentucky Intl,KY,0,0,0,708,0,1996
2,Adams -Dallas Love,Adams,AR,Dallas Love,TX,0,0,0,2605,0,1996
3,Adams -Dallas-Fort Worth International,Adams,AR,Dallas-Fort Worth International,TX,0,0,0,2191,0,1996
4,Adams -Denver Intl,Adams,AR,Denver Intl,CO,0,0,0,97,0,1996


## Now I have a dataframe that includes delay types by states that lets me analyze those delays at the state or airport level!

In [140]:
# i'll save this to a pickle for further analysis
merged.to_pickle('datasets/flight_hist_pickle/cleaned_%d.pickle' % year )